In [1]:
import pandas as pd
from glob import glob
import re,os
import xarray as xr
from typing import Optional, Tuple, Union
from shapely.geometry import Point
import geopandas as gpd

In [2]:
path = '/home/mohammad/Dossier_travail/Raven/test_Q_deh'

In [3]:

meta_patterns = {
    "Station: ": "name",
    "Bassin versant: ": "bv",
    "Coordonnées: (NAD83) ": "coords",
}


data_header_pattern = "Station Date Débit (m³/s) Remarque\n"

In [4]:
os.chdir(path)
files = glob ("*.txt")
dataset = pd.DataFrame(columns = ["station_id"])
latitude = pd.DataFrame(columns={'station_id','lat'},index = None)
longitude = pd.DataFrame(columns={'station_id','lon'},index = None)

for i in range(len(files)): 
    fname = os.path.join(path,files[i])
    print("writing station:",files[i])
    with open(fname, encoding="latin1") as fh:
        txt = fh.read()
        txt = re.sub(" +", " ", txt)
        meta, data = txt.split(data_header_pattern)

    m = dict()
    for key in meta_patterns:
    # Various possible separators to take into account
        m[meta_patterns[key]] = (
            meta.split(key)[1].split(" \n")[0].split("\n")[0].split(" Régime")[0]
            )
    d = pd.read_csv(
    fname,
    delimiter=r"\s+",
    skiprows=len(meta.splitlines()),
    encoding="latin1",
    converters={0: lambda x: str(x)},  # noqa
    index_col=1,
    parse_dates=True,
    infer_datetime_format=True,
)
    d = d.drop(['Remarque','(m³/s)'], axis=1)
    #d=d.dropna()
    d = d.fillna(-1.2345)  # fill NaNs with -1.2345
    if len(d["Station"].unique()) == 1:
        m["station"] = d["Station"].unique()[0]
        d = d.drop("Station", axis=1)
    else:
        raise ValueError("Multiple stations detected in the same file.")
    d = d.rename(columns={"Débit":'qobs'})
    d.index.names = ["time"]
#    d = d.drop("Nan", axis=1)
    
    # finding latitude (in decimal degree)
    coords = m["coords"].split(" // ")  
    deg, minutes, seconds, _ = re.split("[°'\"]", coords[0])

    if float(deg) > 0:
        lat =  round(
            float(deg) + float(minutes) / 60 + float(seconds) / (60 * 60), 6
        )
    else:
        lat =  round(float(deg) - (float(minutes) / 60 + float(seconds) / (60 * 60)), 6)

    # finding longitude (in decimal degree)
    coords = m["coords"].split(" // ")  
    deg, minutes, seconds, _ = re.split("[°'\"]", coords[1])

    if float(deg) > 0:
        lon =  round(
            float(deg) + float(minutes) / 60 + float(seconds) / (60 * 60), 6
        )
    else:
        lon =  round(float(deg) - (float(minutes) / 60 + float(seconds) / (60 * 60)), 6)
    
    d['lon'] = lon
    d['lat'] = lat
    d['station_id'] = m['station']
    
    lt = pd.DataFrame(d,columns={'station_id','lat'},index = None)
    lo = pd.DataFrame(d,columns={'station_id','lon'},index = None)
    d = d.drop("lat", axis=1)
    d = d.drop("lon", axis=1)

    
    dataset = pd.concat([dataset,d],join = 'outer')
    latitude = pd.concat([latitude,lt],join = 'outer')
    longitude = pd.concat([longitude,lo],join = 'outer')
    
    latitude = latitude.drop_duplicates(subset = ["station_id"])
    longitude = longitude.drop_duplicates(subset = ["station_id"])
    
    
    del d,m,coords,lat,lon,deg,minutes,seconds,fname,meta,data,lt,lo

dataset = dataset.rename_axis('time').reset_index()

dataset = dataset.set_index(['station_id', 'time'])
dataset = dataset[~dataset.index.duplicated(keep='first')]

ds = dataset.to_xarray()

writing station: 020601_Q_45A69.txt
writing station: 022601_Q_07A22.txt
writing station: 022703_Q_69A81.txt
writing station: 022704_Q_81A22.txt
writing station: 023002_Q_92A22.txt
writing station: 023004_Q_95A22.txt
writing station: 023106_Q_66A22.txt
writing station: 023303_Q_79A22.txt
writing station: 023401_Q_24A22.txt
writing station: 023402_Q_14A22.txt
writing station: 023403_Q_14A83.txt
writing station: 023422_Q_64A22.txt
writing station: 023426_Q_64A81.txt
writing station: 023432_Q_71A22.txt
writing station: 023702_Q_06A22.txt
writing station: 024003_Q_66A22.txt
writing station: 024007_Q_69A01.txt
writing station: 024010_Q_77A95.txt
writing station: 024013_Q_78A10.txt
writing station: 024014_Q_99A22.txt
writing station: 024015_Q_05A22.txt
writing station: 030101_Q_28A22.txt
writing station: 030103_Q_66A22.txt
writing station: 030215_Q_58A22.txt
writing station: 030219_Q_37A77.txt
writing station: 030234_Q_52A22.txt
writing station: 030238_Q_64A78.txt
writing station: 030242_Q_65

In [5]:
ds

<xarray.Dataset>
Dimensions:     (station_id: 30, time: 39192)
Coordinates:
  * station_id  (station_id) object '020601' '022601' ... '030246' '030282'
  * time        (time) datetime64[ns] 1915-02-27 1915-02-28 ... 2022-06-16
Data variables:
    qobs        (station_id, time) float64 nan nan nan nan ... 7.665 6.276 5.201

In [7]:
ds2 = latitude.set_index(['station_id','lat']).to_xarray()
ds3 = longitude.set_index(['station_id','lon']).to_xarray()

ds2

<xarray.Dataset>
Dimensions:     (station_id: 30, lat: 30)
Coordinates:
  * station_id  (station_id) object '020601' '022601' ... '030246' '030282'
  * lat         (lat) float64 45.28 45.32 45.33 45.34 ... 47.38 47.53 48.93
Data variables:
    *empty*

In [8]:
ds_merged = xr.merge([ds, ds2,ds3])

ds_merged

<xarray.Dataset>
Dimensions:     (station_id: 30, time: 39192, lat: 30, lon: 30)
Coordinates:
  * station_id  (station_id) object '020601' '022601' ... '030246' '030282'
  * time        (time) datetime64[ns] 1915-02-27 1915-02-28 ... 2022-06-16
  * lat         (lat) float64 45.28 45.32 45.33 45.34 ... 47.38 47.53 48.93
  * lon         (lon) float64 -72.31 -72.28 -72.1 ... -69.94 -69.85 -64.64
Data variables:
    qobs        (station_id, time) float64 nan nan nan nan ... 7.665 6.276 5.201

In [ ]:
ds.to_netcdf("SLSO_qobs.nc",'w')